# Imports

In [1]:
import pandas as pd
from pymer4 import Lmer
import warnings
warnings.filterwarnings('ignore')

dat = pd.read_csv('./Data/full_sample_clean.csv', index_col=0)
df = dat.loc[(dat.in_sample == True)&(dat.correct ==True)].reset_index(drop=True).drop('diag', axis=1)

# Response Time

## Between Groups

### RT and Infelicity

In [3]:
import pingouin as pg
ddf =df.loc[(df.in_sample == True) &(df.correct==True)]
print(pg.corr(x=ddf.loc[ddf.group == 'neurotypical'].infelicity.values, y=ddf.loc[ddf.group == 'neurotypical'].rt.values))
print(pg.corr(x=ddf.loc[ddf.group == 'autism'].infelicity.values, y=ddf.loc[ddf.group == 'autism'].rt.values))

            n         r         CI95%    p-val   BF10     power
pearson  4049  0.055712  [0.02, 0.09]  0.00039  10.59  0.943841
            n         r          CI95%     p-val   BF10     power
pearson  2071 -0.035446  [-0.08, 0.01]  0.106829  0.101  0.364451


In [4]:
mod_factors = {'group':df.group.unique().tolist(), 'agent_state':df.agent_state.unique().tolist(),
               'ascription': df.ascription.unique().tolist(), 'item':[str(x) for x in range(1,13)]}
rand_fx = ' + (agent_state + ascription|item) + (agent_state|subject)'

### 3-way interaction effect of Group x Agent State x Ascription
![](Figures/groupxascrip_rt.png)


In [5]:
mb0 = Lmer('rt ~ group*agent_state*ascription + infelicity'+rand_fx, data=df)
mb0.fit(factors=mod_factors, summarize=False, REML=False)
mb1 = Lmer('rt ~ group:agent_state + group:ascription + agent_state:ascription + infelicity'+rand_fx, data=df)
mb1.fit(factors=mod_factors, summarize=False, REML=False)
three_way = Lmer.anova(mb0, mb1)
three_way.iloc[:]

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
0,3.253778e+06,3.253778e+06,1,650.833130,9.586787,0.002044,**
1,4.645477e+05,2.322738e+05,2,14.321286,0.684361,0.520168,
2,1.258738e+07,1.258738e+07,1,14.547306,37.086907,0.000024,***
3,3.765343e+04,3.765343e+04,1,3773.721993,0.110940,0.739095,
4,3.264785e+06,1.632392e+06,2,679.822463,4.809609,0.008430,**
5,7.034607e+04,7.034607e+04,1,4702.301164,0.207265,0.648941,
6,3.559662e+06,1.779831e+06,2,4695.027631,5.244016,0.005310,**
7,2.236641e+06,1.118321e+06,2,4681.989253,3.294970,0.037155,*


#### Result:
Controlling for their isolated effects and all pairwise 2-way interactions, there is a significant 3-way interaction of group, agent state, and ascription, p=.007. Suggesting that:
 - The effect of group on agent state('s effect on RT) is different depending on the ascription type.
OR
 - The effect of group on ascription('s effect on RT) is different depending on the agent state (not likely since group-x-ascription n.s.)
OR
 - The effect of agent state on ascription('s effect on RT), i.e. the agent state x ascription interaction, is different per group.
    - likely driving the overall group x agent state interaction, p=.006

Controlling for their isolated effects & the 3-way interaction above, there's a significant agent state x ascription interaction across groups, p=.00003, suggesting that:
- The effect of agent state on RT differs depending on the kind of ascription for both groups OR
- The effect of ascription type on RT differs depending on the agent state for both groups (more likely)

There is a group difference in over all response time. (p=.002)

Controlling for 2-way interactions with group & agent state, there is a significant effect of ascription type on RT (p=.00001)


### Interaction of Agent State x Ascription
![](Figures/agentxascript.png)

In [6]:
mb2 = Lmer('rt ~ group:agent_state + group:ascription + infelicity'+rand_fx, data=df)
mb2.fit(factors=mod_factors, summarize=False)
agentxascrip = Lmer.anova(mb1, mb2)
agentxascrip.iloc[:]

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
0,3.199344e+04,3.199344e+04,1,3765.940533,0.094092,0.759055,
1,7.093205e+06,1.418641e+06,5,44.777923,4.172206,0.003384,**
2,1.400592e+07,7.002960e+06,2,33.506419,20.595622,0.000001,***
3,6.075622e+06,3.037811e+06,2,4638.907938,8.934166,0.000134,***


#### Result:
 Controlling for infelicity and the interaction of group x agent state and group x ascription type, the interaction of agent state and ascription reduces to marginal significance, p=.05.


### Interaction of Group x Ascription
![](Figures/groupxascript.png)


In [7]:
#mb1 = Lmer('rt ~ group:agent_state + group:ascription + agent_state:ascription + infelicity'+rand_fx, data=df)
mb3 = Lmer('rt ~ group:agent_state + agent_state:ascription + infelicity'+rand_fx, data=df)
mb3.fit(factors=mod_factors, summarize=False)
groupxascrip = Lmer.anova(mb1, mb3)
groupxascrip.iloc[:]

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
0,3.199344e+04,3.199344e+04,1,3765.940533,0.094092,0.759055,
1,7.093205e+06,1.418641e+06,5,44.777923,4.172206,0.003384,**
2,1.400592e+07,7.002960e+06,2,33.506419,20.595622,0.000001,***
3,6.075622e+06,3.037811e+06,2,4638.907938,8.934166,0.000134,***


#### Result:
Controlling for infelicity, the interaction of agent state x group, and the interaction of agent state x ascription, there is a significant interaction between group and ascription. Suggesting that:
- The effect of ascription on RT differs between groups
OR
- The effect of group on RT differs between ascription types

### Interaction of Group x Agent State
![](Figures/agentxgroup.png)


In [8]:
mb4 = Lmer('rt ~ group:ascription + agent_state:ascription + infelicity'+rand_fx, data=df)
mb4.fit(factors=mod_factors, summarize=False)
grpxagent = Lmer.anova(mb1, mb4)
grpxagent.iloc[:]

Model failed to converge with max|grad| = 0.0085698 (tol = 0.002, component 1) 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
0,3.199344e+04,3.199344e+04,1,3765.940533,0.094092,0.759055,
1,7.093205e+06,1.418641e+06,5,44.777923,4.172206,0.003384,**
2,1.400592e+07,7.002960e+06,2,33.506419,20.595622,0.000001,***
3,6.075622e+06,3.037811e+06,2,4638.907938,8.934166,0.000134,***


#### Result:
Controlling for infelicity, the interaction of ascription x group, and the interaction of ascription x agent state, there is a significant interaction of group x agent state. Suggest that:
- The effect of group on RT differs between agent states
OR
- The effect of agent state on RT differs between group

## Within Group - Neurotypical

In [9]:
NT = df.loc[df.group == 'neurotypical']
rand_fx = ' + (agent_state*ascription|item) + (agent_state|subject)'
mod_factors = {'agent_state':NT.agent_state.unique().tolist(), 'ascription': NT.ascription.unique().tolist(), 'item':[str(x) for x in range(1,13)]}

### Agent State x Ascription Interaction - Neurotypical
![](Figures/agentxascript_nt.png)

In [10]:
m0 = Lmer('rt ~ agent_state*ascription + infelicity'+rand_fx, data=NT)
m0.fit(factors=mod_factors, summarize=False)
m1 = Lmer('rt ~ agent_state + ascription + infelicity'+rand_fx, data=NT)
m1.fit(factors=mod_factors, summarize=False)
interaction_effct = Lmer.anova(m0, m1)
interaction_effct.iloc[:]

Model failed to converge with max|grad| = 0.00881185 (tol = 0.002, component 1) 

boundary (singular) fit: see help('isSingular') 

Model failed to converge with max|grad| = 0.00881185 (tol = 0.002, component 1) 

boundary (singular) fit: see help('isSingular') 

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
0,5.097716e+05,2.548858e+05,2,11.941731,0.961018,0.410208,
1,8.530022e+06,8.530022e+06,1,11.113858,32.161496,0.000139,***
2,9.466365e+04,9.466365e+04,1,3281.390631,0.356919,0.550264,
3,1.555735e+06,7.778677e+05,2,10.448258,2.932863,0.097509,.


#### Result:
Controlling for their isolated effects, the interaction of agent state x ascription is significant in neurotypicals,p=.03; suggesting that:
- The effect of agent state on RT is different between ascription types.
OR
- The effect of ascription on RT is different between agent states

### Main Effect of Agent State - Neurotypical
![](Figures/agent_nt.png)

In [11]:
m2 = Lmer('rt ~ ascription + infelicity'+rand_fx, data=NT)
m2.fit(factors=mod_factors, summarize=False)
agent_state_effect= Lmer.anova(m1,m2 )
agent_state_effect.iloc[:]

Model failed to converge with max|grad| = 0.0048822 (tol = 0.002, component 1) 

boundary (singular) fit: see help('isSingular') 

Model failed to converge with max|grad| = 0.00239575 (tol = 0.002, component 1) 

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
0,1.937798e+06,9.688989e+05,2,11.631839,3.660970,0.058445,.
1,1.035652e+07,1.035652e+07,1,10.859509,39.131944,0.000066,***
2,8.366882e+04,8.366882e+04,1,3278.245631,0.316141,0.573974,


#### Result:
There is only a marginally significant effect of agent state in the neurotypical group when controlling for ascription, p=.05.

### Main Effect of Ascription - Neurotypical
![](Figures/ascript_nt.png)

In [12]:
m3 = Lmer('rt ~ agent_state + infelicity'+rand_fx, data=NT)
m3.fit(factors=mod_factors, summarize=False)
ascription_effect = Lmer.anova(m1, m3)
ascription_effect.iloc[:]

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

Model failed to converge with max|grad| = 0.00239575 (tol = 0.002, component 1) 

Model failed to converge with max|grad| = 0.00239575 (tol = 0.002, component 1) 

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
0,1.937798e+06,9.688989e+05,2,11.631839,3.660970,0.058445,.
1,1.035652e+07,1.035652e+07,1,10.859509,39.131944,0.000066,***
2,8.366882e+04,8.366882e+04,1,3278.245631,0.316141,0.573974,


#### Result:
Controlling for the effect of agent state, ascription type has a significant effect on RT in the neurotypical group, p=00009.

## Post-hoc tests of Agent State x Ascription Interaction in Neurotypical group

In [13]:
marginal_estimates, comparisons = m0.post_hoc(marginal_vars='ascription', grouping_vars=['group','agent_state'])
comparisons
#m0.marginal_contrasts

RRuntimeError: Error in emmeans(model, pairwise ~ ascription | group + agent_state, adjust = "tukey",  : 
  No variable named group in the reference grid


## Within Group - Autism

In [ ]:
ASD =  df.loc[df.group == 'autism']

### Agent State x Ascription Interaction - Autism
![](Figures/agentxascript_asd.png)

In [ ]:
m0a = Lmer('rt ~ agent_state*ascription + infelicity'+rand_fx, data=ASD)
m0a.fit(factors=mod_factors, summarize=False)
m1a = Lmer('rt ~ agent_state + ascription + infelicity'+rand_fx, data=ASD)
m1a.fit(factors=mod_factors, summarize=False)
interaction_effct_a = Lmer.anova(m0a, m1a)#.iloc[:]
interaction_effct_a.iloc[:]

#### Result:
Controlling for their isolated effects, there is not a significant interaction between Agent state and ascription type in the autism group.

### Main Effect of Agent State - Autism
![](Figures/agent_asd.png)

In [ ]:
m2a = Lmer('rt ~ ascription + infelicity'+rand_fx, data=ASD)
m2a.fit(factors=mod_factors, summarize=False)
agent_state_effect_a= Lmer.anova(m1a,m2a )
agent_state_effect_a.iloc[:]

#### Result:
Controlling for the effect of ascription, there is a significant effect of agent state on RT in the autism group, p=.04

### Main Effect of Ascription - Autism
![](Figures/ascript_asd.png)

In [ ]:
m3a = Lmer('rt ~ agent_state + infelicity'+rand_fx, data=ASD)
m3a.fit(factors=mod_factors, summarize=False)
ascription_effect_a = Lmer.anova(m1a, m3a)
ascription_effect_a.iloc[:]

#### Result:
Controlling for agent state, there is a significant effect of ascription type on RT in the autism group, p=.00005